In [ ]:
from PacTimeOrig.models import io,NNtorch,training,utils,lossfn
from PacTimeOrig.data import DataHandling as dh
from PacTimeOrig.data import DataProcessing as dp
from PacTimeOrig.Attractor import base as attractor
from PacTimeOrig.utils import processing as proc
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import  matplotlib.pyplot as plt

In [ ]:
#load datafile
datafile=dh.dataloader(sess=2)

#Get session variables
sessvars=dh.getvars(datafile)
sessvars = dh.get_matlab_wt_reaction_time(sessvars,session=2,subj='H')
#Select 2 prey trials
#Get position data
positions=dh.retrievepositions(datafile,rescale = [960.00,540.00])

kinematics,sessvars=dh.subselect(positions,sessvars,trialtype='2')
kinematics=dp.computederivatives(kinematics, vartodiff=['selfXpos','selfYpos','prey1Xpos','prey1Ypos','prey2Xpos','prey2Ypos'], dt=1.0/60.0)
kinematics=dh.dropcols(kinematics, columns_to_drop=['predXpos','predYpos','selfXaccel','selfYaccel','prey1Xaccel','prey1Yaccel','prey2Xaccel','prey2Yaccel'])

kinematics = dh.cut_to_rt(kinematics, sessvars)


Xdsgn=kinematics
for trial in range(len(Xdsgn)):
    Xdsgn[trial]['val1'] = np.repeat(sessvars.iloc[trial].NPCvalA,len(kinematics[trial]))
    Xdsgn[trial]['val2'] = np.repeat(sessvars.iloc[trial].NPCvalB,len(kinematics[trial]))
    
#Switch reward positions so highest value is always in prey 1 slot
Xdsgn=dh.rewardalign(Xdsgn)
Xdsgn = [df[sorted(df.columns)] for df in Xdsgn]


rwds=np.sort(sessvars['NPCvalA'].unique())
for trial in range(len(Xdsgn)):
    #Make one hot columns
    for i in range(len(sessvars['NPCvalA'].unique())):
        Xdsgn[trial]['val1'+str(i+1)]=np.zeros((len(kinematics[trial]),1)).astype(int)
        Xdsgn[trial]['val2'+str(i+1)]=np.zeros((len(kinematics[trial]),1)).astype(int)
    
    Xdsgn[trial]['val1'+str((np.where(Xdsgn[trial].val1[0]==rwds)[0]+1)[0])]=1
    Xdsgn[trial]['val2'+str((np.where(Xdsgn[trial].val2[0]==rwds)[0]+1)[0])]=1
    Xdsgn[trial].drop(columns=['val1','val2'], inplace=True)


## TRAIN ON CONTROL
# Ytrain=list()
# for trial in range(len(Xdsgn)):
#     Ytrain.append(np.vstack([Xdsgn[trial].selfXvel,Xdsgn[trial].selfYvel]).transpose())
#     Xdsgn[trial]=Xdsgn[trial].drop(columns=['selfXvel','selfYvel'],axis=1)


#TRAIN OF FUTURE POS
Ytrain=list()
for trial in range(len(Xdsgn)):
    Ytrain.append(np.vstack([Xdsgn[trial].selfXpos,Xdsgn[trial].selfYpos]).transpose())
    Xdsgn[trial]=Xdsgn[trial].drop(columns=['selfXvel','selfYvel'],axis=1)



Xdsgn=[df.to_numpy() for df in Xdsgn]    
# 
# # #Make from 1:end-1
Xdsgn = [arr[:-1, :] for arr in Xdsgn]
# # 
# # #Make from 2:end
# # 
Ytrain = [arr[1:, :] for arr in Ytrain]


## TRAIN ON CONTROL
# for trial in range(len(Ytrain)):
#     Ytrain[trial]=np.gradient(Ytrain[trial],axis=0)
#     
# Ytrain=proc.smoothing(Ytrain,7)


# This variant predicts and feeds back the position of that predicted by the model.  It fully closes the loop from inputs --> RNN --> RNN --> position 

In [ ]:
dataset = io.VariableLengthDatasetControl(Xdsgn, Ytrain)
dataloader = io.torchloaderControl(dataset,batch_size=12)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NNtorch.TwoAreaRNNNetController(
    input_size=18,
    hidden_size_area1=128,
    hidden_size_area2=128,
    output_size=2,
    dt=16.67,
    tau_area1=100,    # Time constant for Area 1
    tau_area2=20,    # Time constant for Area 2
    alpha_ff=0.1,     # 10% feedforward strength
    alpha_fb=0.05,     # 5% feedback strength
    activation=torch.relu,
    controller_gain=1
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

model,losses=training.train_masked_rnn_derivloss_control(model, dataloader, optimizer, device, lambda_derivative=0.001,num_epochs=30, outcard=2)



trial=0
tmpX=np.delete(Xdsgn[trial],[0,2],axis=1)
tmpX=torch.tensor(tmpX[:-1,:])
seq_len=tmpX.shape[0]
initial_position = torch.tensor([Xdsgn[trial][0,[0,2]][0], Xdsgn[trial][0,[0,2]][1]], dtype=torch.float32)  # Replace x0 and y0 with actual values

seq_lengths = torch.tensor([seq_len], dtype=torch.long).to(device)  # Shape: (1,)
tmpX=tmpX.float()
tmpX = tmpX.unsqueeze(0)  # Now x is of shape (1, seq_len, input_size)
initial_position=initial_position.unsqueeze(0)


model.eval()
with torch.no_grad():
    # Perform inference
    predicted_positions = model(tmpX, seq_lengths, initial_position)  # Shape: (1, seq_len, 2)
    
plt.plot(predicted_positions[0].squeeze(0)[1:,:],'--')
plt.plot(Ytrain[trial][:-1,:])


from sklearn.decomposition import PCA
pca=PCA(4)

plt.plot(pca.fit_transform(predicted_positions[1].squeeze(0)))
plt.show()
plt.plot(pca.fit_transform(predicted_positions[2].squeeze(0)))
plt.show()

In [ ]:
dataset = io.VariableLengthDataset(Xdsgn, Ytrain)
dataloader = io.torchloader(dataset,batch_size=12)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = NNtorch.RNNNetpad_base(input_size=20, hidden_size=128, output_size=2,tau=20,activation=torch.tanh)
# # model = NNtorch.RNNNetpad_latent(input_size=20, hidden_size=64, latent_size=8,output_size=2,tau=20,activation=torch.tanh)
# criterion = nn.MSELoss(reduction='sum')
# # criterion = nn.L1Loss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# # Train the model
# model,losses=training.train_masked_rnn(model, dataloader, criterion, optimizer, device,num_epochs=65,outcard=2)





input_size = 20  # Example input size
hidden_size_area1 = 128
hidden_size_area2 = 128
output_size = 2

model = NNtorch.TwoAreaRNNNet(
    input_size=input_size,
    hidden_size_area1=hidden_size_area1,
    hidden_size_area2=hidden_size_area2,
    output_size=output_size,
    dt=16.67,
    tau_area1=50,    # Time constant for Area 1
    tau_area2=20,    # Time constant for Area 2
    alpha_ff=0.3,     # 10% feedforward strength
    alpha_fb=0.1,     # 5% feedback strength
    activation=torch.tanh
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)


model,losses=training.train_masked_rnn_derivloss(model, dataloader, optimizer, device, lambda_derivative=0.05,num_epochs=30, outcard=2)






# criterion = nn.MSELoss(reduction='sum')
# # criterion = nn.L1Loss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# # Train the model
# model,losses=training.train_masked_rnn(model, dataloader, criterion, optimizer, device,num_epochs=65,outcard=2)


In [ ]:
hidarray = np.zeros((1,model.rnn.hidden_size))
d1list =[]
d2list =[]
for trial in range(150):
    # outputs, hidden,_ = utils.eval_single_trial_pad_rnn(model,Xdsgn[trial])
    # hidarray=np.vstack((hidarray,hidden.detach().numpy()))
    # plt.plot(10*outputs[:,1],'--')
    # plt.plot(Ytrain[trial][:,1])
    # plt.show()
    d1=np.array(np.sqrt(np.power(kinematics[trial].selfXpos-kinematics[trial].prey1Xpos,2)+np.power(kinematics[trial].selfYpos-kinematics[trial].prey1Ypos,2)))
    d2=np.array(np.sqrt(np.power(kinematics[trial].selfXpos-kinematics[trial].prey2Xpos,2)+np.power(kinematics[trial].selfYpos-kinematics[trial].prey2Ypos,2)))
    d1list.append(d1.reshape(-1,1))
    d2list.append(d2.reshape(-1,1))
    
d1list=d1list[2:]
d2list=d2list[2:]
hidarray=hidarray[2:]
scores=pca.fit_transform(hidarray)
